# 유튜브 랭킹 데이터 수집하기

In [ ]:
!apt-get update > /dev/null
!pip install selenium > /dev/null
!apt install chromium-chromedriver > /dev/null

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')   # 화면없이 실행
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome('chromedriver', options=options)

In [ ]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=1'
driver.get(url)
# time.sleep(2)

In [ ]:
trs = driver.find_elements_by_tag_name('tr')
len(trs)

102

In [ ]:
trs = driver.find_elements_by_css_selector('.aos-init')
len(trs)

100

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser') 

In [ ]:
# soup.select('form > table > tbody > tr')
channel_list = soup.select('.aos-init')
len(channel_list)

100

In [ ]:
channel = channel_list[0]
category = channel.select_one('p.category').get_text().strip()
category

'[음악/댄스/가수]'

In [ ]:
name = channel.select_one('.subject a').text.strip()
name

'BLACKPINK'

In [ ]:
subscriber = channel.select_one('.subscriber_cnt').text
view = channel.select_one('.view_cnt').text
video = channel.select_one('.video_cnt').text[:-1]
subscriber, view, video

('381만', '13억7401만', '939')

In [ ]:
channels = []
for channel in channel_list:
  category = channel.select_one('p.category').get_text().strip(' \n[]')
  name = channel.select_one('.subject a').text.strip()
  subscriber = channel.select_one('.subscriber_cnt').text
  view = channel.select_one('.view_cnt').text
  video = channel.select_one('.video_cnt').text[:-1]
  channels.append([category, name, subscriber, view, video])

In [ ]:
df = pd.DataFrame(channels, columns=['카테고리', '채널명', '구독자수', '조회수', '비디오수'])
df.tail()

,카테고리,채널명,구독자수,조회수,비디오수
95,음식/요리/레시피,까니짱 [ G-NI ],408만,10억3471만,482
96,키즈/어린이,로미유 스토리[Romiyu Story],408만,10억6557만,433
97,음악/댄스/가수,NCT DREAM,386만,3억7422만,228
98,BJ/인물/연예인,허팝Heopop,381만,31억3892만,"1,847"
99,애완/반려동물,크림히어로즈,381만,13억7401만,939


* 만과 억을 숫자로 바꿔주는 함수를 쓴다

In [41]:
def convert_unit(s):
  #s = ''.join(s.split('억'))
  s = s.replace('억','').replace('개','').replace(',','')
  s = s.replace('만','0000')
  return f'{int(s):,d}'

In [42]:
convert_unit('10억6557만')

'1,065,570,000'

* 두번째 페이지 - xpath로 찾아 click() 하여 이동하기

In [44]:
# //*[@id="list-skin"]/nav/span/a[1]
# //*[@id="list-skin"]/nav/span/a[2]
driver.find_element_by_xpath('//*[@id="list-skin"]/nav/span/a[1]').click()

In [45]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [77]:
channel_list = soup.select('.aos-init')
len(channel_list)

100

In [74]:
results = []
for channel in channel_list:
    category = channel.select_one('p.category').get_text().strip(' \n[]')
    name = channel.select_one('.subject a').text.strip()
    subscriber = convert_unit(channel.select_one('.subscriber_cnt').text)
    view = convert_unit(channel.select_one('.view_cnt').text)
    video = convert_unit(channel.select_one('.video_cnt').text)
    results.append([category,name,subscriber,view,video])

In [81]:

df = pd.DataFrame(results, columns=['카테고리','채널명','구독자수','조회수','비디오수'])
df.tail()


,카테고리,채널명,구독자수,조회수,비디오수
995,게임,미소,"510,000","210,770,000","3,084"
996,미분류,밉지않은 관종언니,"510,000","67,740,000",170
997,미분류,복지마블TV [Welfare Marble],"510,000","49,960,000",240
998,음식/요리/레시피,Muggo,"510,000","183,720,000","1,388"
999,게임,임선비,"510,000","234,290,000","1,134"


## page 1에서 10까지 크롤링하기

In [80]:
results = []
for page in range(1,11)
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+str(page)
    driver.get(url)
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    channel_list = soup.select('.aos-init')

    for channel in channel_list:
        category = channel.select_one('p.category').get_text().strip(' \n[]')
        name = channel.select_one('.subject a').text.strip()
        subscriber = convert_unit(channel.select_one('.subscriber_cnt').text)
        view = convert_unit(channel.select_one('.view_cnt').text)
        video = convert_unit(channel.select_one('.video_cnt').text)
        results.append([category,name,subscriber,view,video])

In [84]:
df = pd.DataFrame(results, columns=['카테고리', '채널명', '구독자수', '조회수', '비디오수'])
df.tail()

,카테고리,채널명,구독자수,조회수,비디오수
995,게임,미소,"510,000","210,770,000","3,084"
996,미분류,밉지않은 관종언니,"510,000","67,740,000",170
997,미분류,복지마블TV [Welfare Marble],"510,000","49,960,000",240
998,음식/요리/레시피,Muggo,"510,000","183,720,000","1,388"
999,게임,임선비,"510,000","234,290,000","1,134"


In [85]:
df.to_csv('유튜브_순위.csv', index=False)